In [1]:
import numpy as np
import pandas as pd
from tqdm import tqdm

In [2]:
df = pd.read_json('contacts.json')
npdata = df.values
df.head()

,Id,Email,Phone,Contacts,OrderId
0,0,gkzAbIy@qq.com,,1,
1,1,,329442681752,4,vDDJJcxfLtSfkooPhbYnJdxov
2,2,,9125983679,0,
3,3,mdllpYmE@gmail.com,,0,bHquEnCbbsGLqllwryxPsNOxa
4,4,,300364407,2,


### Assume data from column "Email" != data from any other columns

In [3]:
memory = {}
clusters = {}

In [4]:
def add_to_memory(user_id, value):
    if value != "":
        if value in memory:
            memory[value].add(user_id)
            return
        memory[value] = {user_id}

In [5]:
for row in tqdm(npdata):
    user_id = row[0]
    add_to_memory(user_id, row[4])
    add_to_memory(user_id, row[1])
    add_to_memory(user_id, row[2])

100%|███████████████████████████████████████████| 500000/500000 [00:01<00:00, 436115.57it/s]


### Trace only first

In [6]:
for _, ids in tqdm(memory.items(), total=len(memory.items())):
    temp_cluster = set(ids)
    
    # Check if any of these ids have cluster.
    for uid in ids:
        if uid in clusters:
            temp_cluster.update(clusters[uid])
    
    for uid in temp_cluster:
        clusters[uid] = temp_cluster

100%|███████████████████████████████████████████| 629135/629135 [00:01<00:00, 527409.55it/s]


### Then Sum contacts & Build output format

In [7]:
output = []
for user_id, trace in tqdm(sorted(clusters.items()), total=len(clusters.items())):
    contacts = np.sum(npdata[list(trace), 3])
    trace = "-".join([str(trace) for trace in sorted(trace)])
    answer = "{},{}".format(trace, contacts)
    output.append({"ticket_id": user_id,  "ticket_trace/contact": answer})

100%|███████████████████████████████████████████| 500000/500000 [00:04<00:00, 113139.25it/s]


### Trace & Sum

In [8]:
# for _, ids in tqdm(memory.items(), total=len(memory.items())):
#     temp_cluster = set(ids)
    
#     # Check if any of these ids have cluster.
#     for uid in ids:
#         if uid in clusters:
#             temp_cluster.update(clusters[uid]["trace"])
    
#     contacts = np.sum(npdata[list(temp_cluster), 3])
    
#     for uid in temp_cluster:
#         clusters[uid] = {"trace": temp_cluster, "contacts": contacts}

### Build output

In [9]:
# output = []
# for user_id, value in tqdm(sorted(clusters.items()), total=len(clusters.items())):
#     trace = value.get("trace")
#     trace = "-".join([str(trace) for trace in sorted(trace)])
#     answer = "{},{}".format(trace, value.get("contacts"))
#     output.append({"ticket_id": user_id,  "ticket_trace/contact": answer})

In [ ]:
output

In [11]:
output_df = pd.DataFrame(output)
output_df.head(20)

,ticket_id,ticket_trace/contact
0,0,"0,1"
1,1,"1-2458-98519-115061-140081-165605-476346,12"
2,2,"2-159312-322639-348955,4"
3,3,"3,0"
4,4,"4,2"
5,5,5-50-212533-215197-226720-383605-404324-458692...
6,6,"6-38-32871-142067-236367,13"
7,7,"7,1"
8,8,"8-183160-406623,5"
9,9,"9-13-16708-33415-343161-417916-468927-484896,8"


In [ ]:
filename = input("Input filename: ")
output_df.to_csv(filename, index=False)